In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm.notebook import tqdm as tqdm
import pdb
import pandas as pd
import matplotlib.pyplot as plt

In [10]:
class WeatherDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, i):
        # Return row as tensor
        x = torch.tensor(self.data.iloc[i][1:69])
        y = torch.tensor(self.data.iloc[i][69:72])
        return x, y
        # return torch.tensor(self.data.iloc[i])

In [11]:
# Test dataset class
data = WeatherDataset('Inputs-Targets.csv')
dataloader = DataLoader(data, batch_size=4, shuffle=True)
for batch in dataloader:
    x, y = batch
    print(x.shape)
    print(y)
    break

torch.Size([4, 68])
tensor([[5, 4, 5],
        [2, 2, 2],
        [5, 5, 5],
        [1, 2, 2]])


In [12]:
# Define model, resnet18
class WeatherModel(nn.Module):
    def __init__(self):
        super(WeatherModel, self).__init__()
        self.fc1 = nn.Linear(68, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, 3)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

In [14]:
# Train model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = WeatherModel().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 100
losses = []
for epoch in tqdm(range(epochs)):
    for batch in dataloader:
        x, y = batch
        optimizer.zero_grad()
        y_pred = model(x)
        loss = criterion(y_pred, y)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    print(f'Epoch: {epoch}, Loss: {loss.item()}')

# Save model
torch.save(model.state_dict(), 'model.pth')

# Graph losses
plt.plot(losses)
plt.show()

  0%|          | 0/100 [00:00<?, ?it/s]

RuntimeError: mat1 and mat2 must have the same dtype